In [1]:
import numpy as np
import torch
import torch.nn as nn

from torch.utils.data import DataLoader, Dataset

In [2]:
learning_rate = 1e-3

batch_size = 64

training_epochs = 100


In [3]:
filename = 'splitted_dataset.npz'
dataset = np.load(filename)

In [4]:
dataset.files

['inputs_train',
 'inputs_test',
 'inputs_valid',
 'labels_train',
 'labels_test',
 'labels_valid']

In [5]:
inputs_train = dataset['inputs_train']
inputs_test  = dataset['inputs_test']
inputs_valid = dataset['inputs_valid']
labels_train = dataset['labels_train']
labels_test  = dataset['labels_test']
labels_valid = dataset['labels_valid']

In [6]:
class Custom_dataset(Dataset):
    
    def __init__(self, inputs, labels):
        self.inputs = torch.from_numpy(inputs).float()
        self.labels = torch.from_numpy(labels).long()
    
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, index):
        inputs = self.inputs[index]
        labels = self.labels[index]
        return inputs, labels

In [7]:
train_set = Custom_dataset(inputs = inputs_train,
                           labels = labels_train)
test_set  = Custom_dataset(inputs = inputs_test,
                           labels = labels_test)
valid_set = Custom_dataset(inputs = inputs_valid,
                           labels = labels_valid)

In [8]:
train_loader = DataLoader(dataset=train_set,
                          batch_size=batch_size,
                          shuffle=True,
                          drop_last=True)
test_loader  = DataLoader(dataset=test_set,
                          batch_size=batch_size,
                          shuffle=True,
                          drop_last=False)
valid_loader = DataLoader(dataset=valid_set,
                          batch_size=batch_size,
                          shuffle=True,
                          drop_last=False)

In [9]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.Features = nn.Sequential(
            nn.Conv2d(in_channels=4, out_channels=8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3),
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3)
            )
        self.FC1 = nn.Linear(in_features=768, out_features=768, bias=True)
        self.FC2 = nn.Linear(in_features=768, out_features=11, bias=True)
        torch.nn.init.kaiming_normal_(self.FC1.weight)
        torch.nn.init.kaiming_normal_(self.FC2.weight)
        self.Classifier = nn.Sequential(
            self.FC1,
            nn.Dropout(p=0.5),
            self.FC2
            )
    def forward(self, inputs):
        output = self.Features(inputs)
        output = output.view(output.size(0), -1)
        output = self.Classifier(output)
        
        return output

In [10]:
GPU_NUM = 7
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)
print('Currernt cuda device ', torch.cuda.current_device())

if device.type == 'cuda':
    print(torch.cuda.get_device_name(GPU_NUM))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(GPU_NUM)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(GPU_NUM)/1024**3,1), 'GB')

Currernt cuda device  7
GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [11]:
model = CNN().to(device)

In [12]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

------------------------

In [13]:
# Define acc_check function

In [14]:
def acc_check(Net, dataset, epoch):
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataset:
            
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = Net(inputs)
            
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        acc = (100 * correct / total)
        return acc

-----------------------

In [15]:
total_batch = len(train_loader)


for epoch in range(training_epochs):
    avg_cost = 0

    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        hypothesis = model(inputs)
        cost = criterion(hypothesis, labels)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch
    
    test_acc  = acc_check(model, test_loader, epoch)
    train_acc = acc_check(model, train_loader, epoch)
    print('[Epoch : {:>4}] '.format(epoch + 1))
    print('cost = {:>.9} \tAcc_Test  : {:>.3} \tAcc_Train : {:>.3}'.format(avg_cost,test_acc, train_acc))
print('Learning finished')

[Epoch :    1] 
cost = 2.41553211 	Acc_Test  : 7.5 	Acc_Train : 10.3
[Epoch :    2] 
cost = 2.40173554 	Acc_Test  : 13.5 	Acc_Train : 11.6
[Epoch :    3] 
cost = 2.39594626 	Acc_Test  : 11.0 	Acc_Train : 10.9
[Epoch :    4] 
cost = 2.39214087 	Acc_Test  : 8.5 	Acc_Train : 11.3
[Epoch :    5] 
cost = 2.39472747 	Acc_Test  : 11.5 	Acc_Train : 11.2
[Epoch :    6] 
cost = 2.39040732 	Acc_Test  : 11.0 	Acc_Train : 10.7
[Epoch :    7] 
cost = 2.38709807 	Acc_Test  : 13.0 	Acc_Train : 12.6
[Epoch :    8] 
cost = 2.38593602 	Acc_Test  : 12.5 	Acc_Train : 13.7
[Epoch :    9] 
cost = 2.37167287 	Acc_Test  : 12.5 	Acc_Train : 13.2
[Epoch :   10] 
cost = 2.36676788 	Acc_Test  : 12.5 	Acc_Train : 13.4
[Epoch :   11] 
cost = 2.37122369 	Acc_Test  : 13.5 	Acc_Train : 13.3
[Epoch :   12] 
cost = 2.3629899 	Acc_Test  : 15.0 	Acc_Train : 13.4
[Epoch :   13] 
cost = 2.35128903 	Acc_Test  : 12.0 	Acc_Train : 14.2
[Epoch :   14] 
cost = 2.32386994 	Acc_Test  : 10.5 	Acc_Train : 13.7
[Epoch :   15] 
cost = 

In [16]:
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 200 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 200 test images: 35 %


In [17]:
correct = 0
total = 0

with torch.no_grad():
    for data in train_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 200 train images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 200 train images: 98 %
